In [31]:
import time
import os
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [32]:
def baixar_fatura(url, username, password, conta, target_dir, name, servico):
    # Obter o ano e mês atuais
    now = datetime.now()
    ano_atual = now.year
    mes_atual = now.strftime('%m')
    
    # Procurar a pasta que contenha o nome (name)
    user_dir = None
    for root, dirs, files in os.walk(target_dir):
        for dir_name in dirs:
            if name in dir_name:
                user_dir = os.path.join(root, dir_name)
                break
    
    # Se a pasta do usuário não existir, criar
    if not user_dir:
        user_dir = os.path.join(target_dir, name)
        os.makedirs(user_dir, exist_ok=True)
    
    # Criar os diretórios ano, ALGAR, serviço e mês se não existirem
    final_dir = os.path.join(user_dir, str(ano_atual), 'ALGAR', servico, mes_atual)
    os.makedirs(final_dir, exist_ok=True)
    
    # Verificar se o arquivo já existe
    file_path = os.path.join(final_dir, f"{conta}.pdf")
    if os.path.exists(file_path):
        print(f"Fatura de {name} da conta {conta} já foi salvo no caminho {file_path}")
        return
    
    # Configurar opções do Chrome para definir o diretório de download
    chrome_options = Options()
    prefs = {'download.default_directory': final_dir}
    chrome_options.add_experimental_option('prefs', prefs)
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    # Preencher o campo de username
    username_input = driver.find_element(By.ID, "username")
    username_input.send_keys(username)
    
    # Preencher o campo de password
    password_input = driver.find_element(By.ID, "password")
    password_input.send_keys(password)
    
    # Clicar no botão de login
    login_button = driver.find_element(By.CLASS_NAME, "_button_p1hl9_1")
    login_button.click()
    
    # Aguardar até que a página de faturas esteja carregada
    WebDriverWait(driver, 1000).until(
        EC.presence_of_element_located((By.CLASS_NAME, "styles__cardInvoices___2S2_n"))
    )
    driver.get('https://algartelecom.com.br/AreaClienteCorporativo/invoices')

    # Verificar se o elemento com id 'account-billing-switcher' existe em 10 segundos, se não, dá um reload na página (F5)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "account-billing-switcher"))
        )
    except TimeoutException:
        driver.refresh()
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, "account-billing-switcher"))
        )

    # Validar se o valor do input é igual à variável 'conta'
    account_input = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.ID, "account-billing-switcher"))
    )
    
    # Verificar se o valor do input é igual à variável 'conta'
    is_equal = account_input.get_attribute('value') == conta
    if not is_equal:
        account_input.click()
        WebDriverWait(driver, 100).until(
            EC.presence_of_element_located((By.ID, 'account-billing-switcher__listbox'))
        )
        listbox = driver.find_element(By.ID, 'account-billing-switcher__listbox')
        options = listbox.find_elements(By.TAG_NAME, 'li')
        for option in options:
            if conta in option.text:
                option.click()
                break
                
    # Se o valor for igual, clicar no botão de download
    is_equal = account_input.get_attribute('value') == conta
    if is_equal:
        try:
            # Adicionar espera explícita para garantir que o elemento esteja presente
            span_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//section[@class="box__content___3L92u"]//div[@class="box__viewport___1ECFm"]//p[@class="styles__loader___3HNfI"]//span[text()="Você não possui faturas a vencer nessa conta de faturamento."]'))
            )
            # Se o elemento for encontrado, notifica que a fatura já foi paga
            if span_element:
                print(f"Fatura de {name} da conta {conta} já foi paga")
        except (NoSuchElementException, TimeoutException):
            # Se o elemento não for encontrado ou ocorrer um timeout, continua para o bloco de download
            download_button = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.XPATH, '//section[@class="box__content___3L92u"]//table[@class="styles__table___231kO"]//td[@class="styles__actions___1N_GR"]//button[@type="button" and contains(@class, "button-image__buttonImage___1bZXf")]'))
            )
            download_button.click()
            # Esperar até que o botão "2ª via" esteja presente e clicar nele
            segunda_via_button = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.XPATH, '//button[contains(text(), "2ª via")]'))
            )
            segunda_via_button.click()
            time.sleep(10)
            
            # Renomear o arquivo baixado para o nome da conta
            downloaded_file = max([os.path.join(final_dir, f) for f in os.listdir(final_dir)], key=os.path.getctime)
            os.rename(downloaded_file, file_path)

            #Notifica o caminho salvo
            print(f"Fatura de {name} da conta {conta} salvo no {file_path}")
    
    driver.quit()
    
# Passe os dados da fatura que deseja coletar seguindo o modelo abaixo
contas = [
    {'name': 'base 1', 'username': '11111111111111', 'password': '999999999999', 'conta': '111111111111', 'servico': 'Telefonia'},
    {'name': 'Campinas', 'username': '22222222222222', 'password': '8888888888888', 'conta': '222222222222', 'servico': 'Internet'},
]

url = 'https://algartelecom.com.br/AreaCliente/autenticacao'

# Passe a raiz do caminho que deseja salvar
target_dir = ''


# Loop para baixar as faturas de todas as contas
for conta in contas:
    baixar_fatura(url, conta['username'], conta['password'], conta['conta'], target_dir, conta['name'], conta['servico'])